# Tratar dados de Compradores e Ofertas Hotmart

### Objeto: Isolar a lista de Ofertas das campanhas lcto-bf25-leads e lcto-bf25-alunos e isolar respectivos compradores

In [1]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df
)

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


## Load Dataframes


### tratamento das Compras Hotmart

In [2]:
raw_sales_approved = hotmart.get_sales_history(
    start_date='2025-11-03',
    transaction_status='APPROVED',
    product_id='3310674'
)

df_hotmart = flatten_list_to_df(raw_sales_approved)

In [5]:
raw_sales_approved

[{'producer': {'ucode': '1c4384b6-3cd8-47e9-bc94-96a7d6e713fc',
   'name': 'Matheus Werner'},
  'product': {'id': 3310674, 'name': 'Cronograma dos Fluentes '},
  'buyer': {'ucode': 'e939ad81-4fcf-4674-bc1b-85411317a3e6',
   'email': 'luiscalcada1998@gmail.com',
   'name': 'Luís Filipe Lima Araújo'},
  'purchase': {'order_date': 1762215357000,
   'approved_date': 1762215388000,
   'commission_as': 'PRODUCER',
   'price': {'currency_code': 'EUR', 'value': 504.3},
   'payment': {'installments_number': 1,
    'method': 'UNKNOWN',
    'type': 'BILLET'},
   'is_subscription': False,
   'transaction': 'HP0243318796',
   'tracking': {'source_sck': 'utm_source=youtube&utm_campaign=lcto-bf25-leads&utm_content=chat&utm_medium=live&campaign_id=lcto-bf25-leads'},
   'offer': {'payment_mode': 'UNIQUE_PAYMENT', 'code': 'aflr0gju'},
   'warranty_expire_date': 1763424000000,
   'status': 'APPROVED',
   'hotmart_fee': {'base': 410,
    'total': 22.88,
    'percentage': 5,
    'fixed': 0.58,
    'currenc

In [8]:
df_hotmart = load_raw_data('hotmart.csv', sep=';')

def clean_data(hotmart_transactions):
    # Drop columns: 'Nome do Produtor', 'Documento do Produtor' and 49 other columns
    hotmart_transactions = hotmart_transactions.drop(columns=['Nome do Produtor', 'Documento do Produtor', 'Nome do Afiliado', 'Transação', 'Meio de Pagamento', 'Origem', 'Moeda', 'Preço do Produto', 'Moeda.1', 'Preço da Oferta', 'Taxa de Câmbio', 'Moeda.2', 'Preço Original', 'Número da Parcela', 'Data de Venda', 'Data de Confirmação', 'Status', 'Documento', 'CEP', 'Cidade', 'Estado', 'Bairro', 'País', 'Endereço', 'Número', 'Complemento', 'chave', 'Código do Produto', 'Código da Afiliação', 'Tipo de Pagamento', 'Período Grátis', 'Tem co-produção', 'Venda feita como', 'Preço Total', 'Tipo pagamento oferta', 'Taxa de Câmbio Real', 'Preço Total Convertido', 'Quantidade de itens', 'Oferta de Upgrade', 'Cupom', 'Moeda.3', 'Valor que você recebeu convertido', 'Data Vencimento', 'Taxa de Câmbio do valor recebido', 'Instagram', 'Origem da venda', 'Moeda de recebimento', 'Faturamento líquido', 'Código do assinante', 'Nota Fiscal', 'Valor do frete bruto'])
    # Filter rows based on column: 'Recorrência'
    hotmart_transactions = hotmart_transactions[(hotmart_transactions['Recorrência'] == 1) | (hotmart_transactions['Recorrência'].isna())]
    hotmart_transactions['DDD'] = process_phone_string(hotmart_transactions, 'DDD')
    hotmart_transactions['Telefone'] = process_phone_string(hotmart_transactions, 'Telefone')
    hotmart_transactions['phone'] = hotmart_transactions.apply(lambda x: x['Telefone'] if pd.isna(x['DDD']) else f'55{x["DDD"]}{x["Telefone"]}', axis=1)
    # Drop column: 'DDD'
    hotmart_transactions = hotmart_transactions.drop(columns=['DDD', 'Telefone'])

    # Drop column: 'Recorrência'
    hotmart_transactions = hotmart_transactions.drop(columns=['Recorrência'])
    # Rename column 'Nome do Produto' to 'product_name'
    hotmart_transactions = hotmart_transactions.rename(columns={'Nome do Produto': 'product_name', "Nome": "name", "Email": "email","Código de Oferta": "offer_id" })
    # Drop column: 'Origem de Checkout'
    hotmart_transactions = hotmart_transactions.drop(columns=['Origem de Checkout'])
    hotmart_transactions = hotmart_transactions[hotmart_transactions['product_name'] == 'Cronograma dos Fluentes ']
    return hotmart_transactions

df_hotmart = clean_data(df_hotmart.copy())
df_hotmart

,product_name,name,email,offer_id,phone
0,Cronograma dos Fluentes,Janaina Mesquita,janaina9cazassa@gmail.com,zgvidddk,19784931716
1,Cronograma dos Fluentes,Eduardo Mendonça da fontoura,edumendonca87@gmail.com,pj5p39qx,5555999857198
2,Cronograma dos Fluentes,Tais Gabriela de Siqueira Santos Silva,tais.3s@hotmail.com,68uw6eyv,5511943384147
3,Cronograma dos Fluentes,Armando Ostler,ostlerarmando@gmail.com,99hdhw5x,5512991615565
4,Cronograma dos Fluentes,Melkzedekue de Moraes Alcântara Calabrese Moreira,sdalcantara@yahoo.com.br,99hdhw5x,5516988091940
...,...,...,...,...,...
2899,Cronograma dos Fluentes,Eduardo Ferreira Santana,eduardof.santana@outlook.com,igxwngei,5511967016960
2900,Cronograma dos Fluentes,ALANA AUGUSTA LISBOA,alana@atuarassessoria.com.br,igxwngei,5531985154623
2901,Cronograma dos Fluentes,Thiago julio,thiagojulio20@gmail.com,igxwngei,5548998551473
2902,Cronograma dos Fluentes,Marcelo Araújo e Silva,marceloas78@gmail.com,igxwngei,5561981861510


### Tratamento das compras TMB

In [9]:
df_tmb = load_raw_data('tmb.csv', sep=';')

"""
Cell generated by Data Wrangler.
"""
def clean_data(df_tmb):
    # Drop columns: 'Produtor', 'Pedido' and 29 other columns
    df_tmb = df_tmb.drop(columns=['Produtor', 'Pedido', 'Endereço completo', 'Logradouro', 'Número', 'Bairro', 'Complemento', 'CEP', 'Estado', 'Cidade', 'País', 'Status', 'Status Financeiro', 'Status Cancelamento', 'Ticket (R$)', 'Modalidade de Contrato', 'Criado Em', 'Data Efetivado', 'Data Cancelado', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_content', 'utm_last_source', 'utm_last_medium', 'utm_last_campaign', 'utm_last_content', 'Renovação automatica', 'Data de Renovação Prevista', 'Renovação enviada', 'Oferta'])
    # Drop column: 'Cliente CPF'
    df_tmb = df_tmb.drop(columns=['Cliente CPF'])
    # Rename column 'Produto' to 'offer_code'
    df_tmb = df_tmb.rename(columns={'Produto': 'offer_id', "Cliente Nome": "name", "Cliente Email": "email", "Telefone": "phone"})

    df_tmb['phone'] = process_phone_string(df_tmb, 'phone')
    
    return df_tmb

df_tmb = clean_data(df_tmb.copy())
df_tmb.head()

,offer_id,name,email,phone
0,COM-BF25-Oferta Negociação,Edion Alberto Torres,edion.torres@poupex.com.br,5561985275734
1,COM-BF25-Oferta Live,Jorge Cabral Ribeiro Bessa,jorgebessa.br@gmail.com,5521999811560
2,COM-BF25-Oferta Pós-Live,Felipe Mendes De Souza,felix153301@gmail.com,5511979908127
3,COM-BF25-Oferta Negociação,ELIAS MAURO DECHAMP VIDAL JUNIOR,jrcuiacap@hotmail.com,5542999218584
4,COM-BF25-alunos-Oferta Live,Fernanda Rabelo da Silva,feerabelo@outlook.com,5511976100429


### Merge nos dados de Compradores

In [10]:
df_buyers = pd.concat([df_hotmart, df_tmb], ignore_index=True)
df_buyers.head()

,product_name,name,email,offer_id,phone
0,Cronograma dos Fluentes,Janaina Mesquita,janaina9cazassa@gmail.com,zgvidddk,19784931716
1,Cronograma dos Fluentes,Eduardo Mendonça da fontoura,edumendonca87@gmail.com,pj5p39qx,5555999857198
2,Cronograma dos Fluentes,Tais Gabriela de Siqueira Santos Silva,tais.3s@hotmail.com,68uw6eyv,5511943384147
3,Cronograma dos Fluentes,Armando Ostler,ostlerarmando@gmail.com,99hdhw5x,5512991615565
4,Cronograma dos Fluentes,Melkzedekue de Moraes Alcântara Calabrese Moreira,sdalcantara@yahoo.com.br,99hdhw5x,5516988091940


### Tratar dados de Oferta

In [11]:
df_offers = load_raw_data('offers.csv')

"""
Cell generated by Data Wrangler.
"""
def clean_data(df_offers):
    # Drop columns: 'updated_at', 'created_at' and 13 other columns
    df_offers = df_offers.drop(columns=['updated_at', 'created_at', 'status', 'description', 'start_date', 'end_date', 'price', 'main_product_id', 'access_duration_months', 'is_default', 'entry_value', 'installment_12x_value', 'is_child', 'comercial_use', 'price_type'])
    # Filter rows based on column: 'campaign_id'
    df_offers = df_offers[df_offers['campaign_id'].str.contains("lcto-bf25", regex=False, na=False, case=False)]
    return df_offers

df_offers = clean_data(df_offers.copy())
df_offers.head()

,offer_id,name,campaign_id,platform,base_url
7,COM-BF25-alunos-Oferta Downsell,[TMB][BF25A] Oferta Downsell\t,lcto-bf25-alunos,TMB,https://pay.tmbeducacao.com.br/MatheusWerner/Z...
12,aflr0gju,[BF25] Oferta Live - v2,lcto-bf25-leads,Hotmart,https://pay.hotmart.com/M86482747L?off=aflr0gju
15,i2jmb51w,[COM][BF25A] Oferta Pós-Live 2 - Filha,lcto-bf25-alunos,Hotmart,https://pay.hotmart.com/M86482747L?off= i2jmb51w
23,7ne4llz3,[COM][BF25] Oferta Pós-Live,lcto-bf25-leads,Hotmart,https://pay.hotmart.com/M86482747L?off=7ne4llz...
24,selbsff3,[COM] [BF25] Upsell Oferta 1,lcto-bf25-alunos,Hotmart,https://pay.hotmart.com/Q92997304O?off=selbsff...


# Realizando Merge no Dataframes


In [12]:
df_buyers_offers = pd.merge(df_buyers, df_offers, on='offer_id', how='left')